# Introduction to crawlers/spiders in Python

This notebook contains a short introduction to working with crawlers/spiders with `Scrapy`:

- What are crawlers/spiders?
- Building a simple crawler using `Scrapy`

## What are crawlers/spiders?

Where "web scraping" refers to (mostly) automated collections of data and material from websites, crawlers and spiders are bots/programs specifically developed to traverse several websites and perform some scraping tasks.

If we are interested in scraping the content of several websites without knowing the exact URLs of those websites, a crawler can be used to go from site to site and perform the necessary web scraping task.

Developing crawlers can be especially tricky if they have to traverse several domains. This is because the web is connected in such a way where a few sites are dominant and are linked to across most websites (just think of how often you see links to Google, Twitter, Facebook etc. on a website). Imagining the web as an ocean with layers like the figure below, a crawler will always move towards the surface because the websites located there are referenced so often.

Obviously we want to avoid the surface with a crawler, as it will then end up trying to crawl the entire web.

![websea](../img/web_sea.png)

*Source unknown*

### Constructing a crawler

The following should be considered when constructing a crawler:
- Where should the crawler start?
- What sites are of interest?
- What scraping task should the crawler do?
- How should the crawler be limited?

In Python, the best way of constructing a crawler is to use relevant data structures to define the starting points and possible sites to avoid. The scraping tasks can be defined as functions to be integrated in the crawler.

## Building a scraper (using `Scrapy`)

The package [`scrapy`](https://docs.scrapy.org/en/latest/) is used for various web scraping purposes. 

One major challenge when crawling is the massive amount of request-handling needed to crawl across various site (the crawler has to keep sending new requests and not just stop if it encounters a timeout). Another thing to be aware of is crawler-restrictions on the page (`robots.txt`) and avoiding sending too many requests to a server too quickly.

Luckily `scrapy` has a lot of existing functions and classes that are created to account for common problems in scraping. Using scrapy, one can focus on the actual scraping tasks that needs to be performed.

Here is a boiled down version of how to create a simple scraper using `scrapy`:
- Create a crawler-class that is adapted from the base class `scrapy.Spider` (fx `my_crawler`)
    - Name the spider by creating a `name` attribute (this is used to call it later)
    - Specify the URLs to scrape in a `start_urls` attribute
    - (Optional) Specify how the scraper should initially process the URLs in `start_urls` (by default, it sends a GET request for each and returns a response object)
    - Specify how each response from the requests send should be processed by defining a `parse` function
- Create a data structure for the scraped info to be stored in
- Call the `CrawlerProcess()` from `scrapy`: `process = CrawlerProcess()`
- Define what crawler the `CrawlerProcess()` should use: `process.crawl(my_crawler)`
- Start the crawling: `process.start()`

**NOTE ON RESTARTING CRAWLERS**

A spacy crawler can only be run once in a given notebook instance. To restart the crawler, you have to restart the kernel of the notebook as well.

In [1]:
import requests
import scrapy
from scrapy.crawler import CrawlerProcess
from urllib.parse import urljoin
from bs4 import BeautifulSoup as bs

ModuleNotFoundError: No module named 'scrapy'

In [2]:
class eu_crawler(scrapy.Spider): #intentional error to avoid mass crawling
    name = "eu_crawler"
    main_url = 'https://ec.europa.eu/clima/news-your-voice/news_en'
    start_urls = ['https://ec.europa.eu/clima/news-your-voice/news_en']
    
    def parse(self, response):
        soup = bs(response.text, "html.parser") # Notice that HTML content is refered to as .text in a scrapy response
        
        article_rows_soup = soup.find_all("article", class_ = "ecl-content-item")
        
        for row in article_rows_soup:
            article_dict = {}

            article_title_soup = row.find("div", class_ = "ecl-content-item__title").find("a")
            article_title = article_title_soup.get_text()
            article_link = article_title_soup['href']

            article_date = row.find("time")["datetime"]

            article_summary_soup = row.find("div", class_ = "ecl-content-item__description")
            try:
                article_summary = article_summary_soup.get_text(strip = True)
            except:
                article_summary = ""

            article_dict['title'] = article_title
            article_dict['link'] = article_link
            article_dict['date'] = article_date
            article_dict['summary'] = article_summary

            article_list.append(article_dict)
        
        try:
            next_page_url = urljoin(self.main_url, soup.find("a", attrs = {'aria-label': "Go to next page"})['href'])
        except:
            next_page_url = None
            
        if next_page_url is not None:
            yield scrapy.Request(url = next_page_url, callback=self.parse)

article_list = []
process = CrawlerProcess(
    {'USER_AGENT': 'Mozilla/5.0'}
)
process.crawl(eu_crawler)
process.start()

2022-10-06 16:03:52 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: scrapybot)
2022-10-06 16:03:52 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:35:26) [GCC 10.4.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.173.el7-x86_64-with-glibc2.36
2022-10-06 16:03:52 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0'}
2022-10-06 16:03:52 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-10-06 16:03:52 [scrapy.extensions.telnet] INFO: Telnet Password: e9fbfcc3a2812c59
2022-10-06 16:03:53 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-10-06 16:03:53 [scrapy.middleware] INFO: Enab

In [4]:
import pandas as pd

df = pd.DataFrame.from_records(article_list)

In [5]:
df.head()

,title,link,date,summary
0,More signatories join the EU Mission for Adapt...,/news-your-voice/news/more-signatories-join-eu...,2022-09-29T12:00:00Z,100 more regions and local authorities sign th...
1,CO2 emissions from new cars in Europe plummete...,/news-your-voice/news/co2-emissions-new-cars-e...,2022-09-26T12:00:00Z,Average CO2 emissions of new cars registered i...
2,Climate Diplomacy Weeks 2022,/news-your-voice/news/climate-diplomacy-weeks-...,2022-09-19T12:00:00Z,"From 12 September to 30 October, EU Delegation..."
3,Breathing LIFE into the fight against climate ...,https://ec.europa.eu/clima/news-your-voice/sto...,2022-09-19T12:00:00Z,Summer 2022 was Europe’s hottest on record – a...
4,Second EU Innovation Fund call for small-scale...,/news-your-voice/news/second-eu-innovation-fun...,2022-09-08T12:00:00Z,In response to the second EU Innovation Fund c...


In [6]:
df.shape

(907, 4)